In [ ]:
import youtube_dl
import requests
from __future__ import unicode_literals

### Download ffmpeg
This demo requires ffmpeg to extract frames and create a new video that is enhanced by `single-image-super-resolution-1033` from the [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo). Download and install for Windows, MacOS or Linux here: http://ffmpeg.org/download.html

In [ ]:
# Use youtube-dl to download the sample video
ydl_opts = {'outtmpl': './original_1080p.mp4'}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://youtu.be/V8yS3WIkOrA'])

In [ ]:
# Make a directory for the video frames
!mkdir frames

In [ ]:
# Extract 640x360 frames from the original video file
!ffmpeg -i original_1080p.mp4 -vf "scale=640:360,fps=30" frames/sr_%04d.png

In [ ]:
### Placeholder for C++ demo app compiling instructions
###
### !export DYLD_LIBRARY_PATH=/opt/intel/openvino_2021/deployment_tools/ngraph/lib/

In [ ]:
def download_model(url):
    local_filename = url.split('/')[-1]

    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filename

In [ ]:
# Download the models 
url = "https://storage.openvinotoolkit.org/repositories/open_model_zoo/2021.3/models_bin/1/single-image-super-resolution-1033/FP16-INT8/"
xml_file = 'single-image-super-resolution-1033.xml'
bin_file = 'single-image-super-resolution-1033.bin'

download_model(url + xml_file)
download_model(url + bin_file)

In [ ]:
# Loop through each frame and enhance with super resolution model (not efficient, will find a better way to do this)
# This requires the compiled Linux/Mac or Windows C++ binary "super_resolution_demo" from OMZ demos
!{for FILE in frames/*; do ./super_resolution_demo -i $FILE -m ./single-image-super-resolution-1033.xml; done} &> /dev/null

In [ ]:
# Create a new video with the frames enhanced by single-image-super-resolution-1033
!ffmpeg -framerate 30 -i frames/sr_%04d.png.png -vcodec libx264 -s 1920x1080 -pix_fmt yuv420p enhanced_1080.mp4